<a href="https://colab.research.google.com/gist/analyticsindiamagazine/8a9015b7f8d12d5b414e89007d9b069a/starter_notebook_for_participants.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import warnings
warnings.filterwarnings('ignore')

import gc
import pickle  as pk
import os
import warnings
warnings.filterwarnings('ignore')

# Modules for data manipulation
import numpy as np
import pandas as pd
import re

# Modules for visualization
import matplotlib.pyplot as plt
import seaborn as sb

# Tools for preprocessing input data
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
# Tools for creating ngrams and vectorizing input data
# import gensim

from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser



# Tools for building a model
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer



from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Bidirectional,Flatten
from keras.layers.embeddings import Embedding

from keras.preprocessing import text
from keras.preprocessing.sequence import pad_sequences

# Tools for assessing the quality of model prediction
from sklearn.metrics import accuracy_score, confusion_matrix,log_loss,f1_score


import tensorflow_hub as hub




Using TensorFlow backend.


## Import Required Files

In [1]:
import pandas as pd
import numpy as np
import os
import pickle as pk

In [2]:
## change it to the unzip path of the downloaded dataset..
data_folder = r'Scripts'

In [3]:
all_files = os.listdir(data_folder)
print('Total Number of Files :', len(all_files))

Total Number of Files : 2827


# Read Train Files

In [4]:
train_df = pd.read_csv('Train.csv')
train_df.head()

,File_Name,Labels
0,file_2180.txt,8
1,file_693.txt,4
2,file_2469.txt,6
3,file_2542.txt,6
4,file_378.txt,16


In [5]:
train_df.Labels.value_counts()

6     405
19    261
4     243
0     203
5     141
15    134
1     116
16    109
11    104
8      79
14     75
7      27
2      25
20     18
13     15
21      9
12      4
9       3
3       2
17      2
10      2
18      1
Name: Labels, dtype: int64

In [11]:
train_df[train_df.Labels == 9 ]

,File_Name,Labels
1078,file_1091.txt,9
1332,file_1092.txt,9
1857,file_1090.txt,9


In [14]:
# open('Scripts/file_1092.txt','r').read()

In [15]:
test_df = pd.read_csv('Test.csv')
test_df.head()

,File_Name
0,file_2300.txt
1,file_809.txt
2,file_1383.txt
3,file_983.txt
4,file_1713.txt


In [8]:
## let's read the text scripts in the train and test dataframes..

train_df['Script'] = [open(data_folder + os.sep + file, "r").read() for file in train_df['File_Name']]
test_df['Script'] = [open(data_folder + os.sep + file, "r").read() for file in test_df['File_Name']]

In [9]:
len(train_df)

1978

In [11]:
pk.dump(train_df,open('train_x','wb'))
pk.dump(test_df,open('test_x','wb'))

# Lets look at a script file after Reading..

In [14]:
#lets check one of the scripts..
print(train_df['Script'][4][:3000])

<b><!--

</b>if (window!= top)

top.location.href=location.href

<b>// -->

</b>

The Abyss - by James Cameron 

                                  THE ABYSS

                            AN ORIGINAL SCREENPLAY

                                      BY

                                JAMES CAMERON

                               August 2, 1988

                             Director's Revision

------------------------------------------------------------------------------

                                  THE ABYSS

OMITTED                                                                 1

OMITTED                                                                 2

TITLE: THE ABYSS -- ON BLACK, DISSOLVING TO COBALT BLUE

EXT. OCEAN/UNDERWATER -- DAY                                            3

Blue, deep and featureless, the twilight of five hundred feet down.

PROPELLER SOUND.  Materializing out of the blue limbo is the enormous but

sleek form of an Ohio-class SSBN ballistic missile su

In [0]:
print(test_df['Script'][4][:3000])

                                          KING KONG

                                          Written by

                        Fran Walsh, Philippa Boyens and Peter Jackson

                                     Based on a Story by

                             Merian C. Cooper and Edgar Wallace

                                                                   1.

          EXT. CENTRAL PARK - DAY

          CLOSE ON: A scrawny MONKEY scratches.

          ANGLES ON: Defeated, listless ANIMALS, in the bleak environs of a

          dilapidated ZOO.

          WIDER: It is CENTRAL PARK ZOO in depression era NEW YORK. The PARK

          itself is like a GARBAGE DUMP, dotted with squalid SHANTY TOWNS.

          Against these BLEAK IMAGES, the SOUND of a BRIGHT, BRASSY SONG

          fades up: Al Jolson, singing "I'm Sitting on Top of the World".

          The sky line of MANHATTAN rises in the background, a grim steaming

          jungle on this cold FALL day.

                 

In [15]:
train_df.Labels.value_counts()

6     405
19    261
4     243
0     203
5     141
15    134
1     116
16    109
11    104
8      79
14     75
7      27
2      25
20     18
13     15
21      9
12      4
9       3
3       2
17      2
10      2
18      1
Name: Labels, dtype: int64

In [1]:
import pickle as pk
train_data = pk.load(open('processed_train_data','rb'))
test_data = pk.load(open('processed_test_data','rb'))


NameError: name 'pk' is not defined

### There is single instance for one class (Label == 18), lets duplicate the row for stratification

In [16]:
train_df = train_df.append(train_df[train_df['Labels'] == 18])
train_df.reset_index(drop=True, inplace=True)

In [17]:
train_df.Labels.value_counts()

6     405
19    261
4     243
0     203
5     141
15    134
1     116
16    109
11    104
8      79
14     75
7      27
2      25
20     18
13     15
21      9
12      4
9       3
3       2
17      2
18      2
10      2
Name: Labels, dtype: int64

# Import the Modeling Libraries 

In [18]:
import numpy as np
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords


Using TensorFlow backend.


Let's load the datasets

In [19]:
# nltk.download('stopwords')
# stop_words = stopwords.words('english')

# Define the Scoring Metric

In [20]:
from sklearn.metrics import log_loss

In [79]:
from nltk import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
from string import punctuation
import re
from nltk.stem import PorterStemmer

stem = PorterStemmer()

extended = ["a","a's","able","about","above","according","accordingly","across","actually","after",
                                "afterwards","again","against","ain't","all","allow","allows","almost","alone","along",
                                "already","also","although","always","am","among","amongst","an","and","another","any",
                                "anybody","anyhow","anyone","anything","anyway","anyways","anywhere","apart",
                                "appear","appreciate","appropriate","are","aren't","around","as","aside",
                                "ask","asking","associated","at","available","away","awfully","b","be",
                                "became","because","become","becomes","becoming","been","before","beforehand",
                                "behind","being","believe","below","beside","besides","best","better","between","beyond",
                                "both","brief","but","by","c","c'mon","c's","came","can","can't","cannot","cant","cause","causes","certain","certainly",
                                "changes","clearly","co","com","come","comes","concerning","consequently","consider","considering","contain","containing",
                                "contains","corresponding","could","couldn't","course","currently","d","definitely","described","despite","did","didn't",
                                "different","do","does","doesn't","doing","don't","done","down","downwards","during","e","each","edu","eg","eight","either",
                                "else","elsewhere","enough","entirely","especially","et","etc","even","ever","every","everybody","everyone","everything","everywhere",
                                "ex","exactly","example","except","f","far","few","fifth","first","five","followed","following","follows","for","former",
                                "formerly","forth","four","from","further","furthermore","g","get","gets","getting","given","gives","go","goes","going",
                                "gone","got","gotten","greetings","h","had","hadn't","happens","hardly","has","hasn't","have","haven't","having","he","he's",
                                "hello","help","hence","her","here","here's","hereafter","hereby","herein","hereupon","hers","herself","hi","him","himself","his",
                                "hither","hopefully","how","howbeit","however","i","i'd","i'll","i'm","i've","ie","if","ignored","immediate","in","inasmuch","inc",
                                "indeed","indicate","indicated","indicates","inner","insofar","instead","into","inward","is","isn't","it","it'd","it'll","it's","its",
                                "itself","j","just","k","keep","keeps","kept","know","known","knows","l","last","lately","later","latter","latterly","least","less",
                                "lest","let","let's","like","liked","likely","little","look","looking","looks","ltd","m","mainly","many","may","maybe","me","mean",
                                "meanwhile","merely","might","more","moreover","most","mostly","much","must","my","myself","n","name","namely","nd","near","nearly",
                                "necessary","need","needs","neither","never","nevertheless","new","next","nine","no","nobody","non","none","noone","nor","normally",
                                "not","nothing","novel","now","nowhere","o","obviously","of","off","often","oh","ok","okay","old","on","once","one","ones","only",
                                "onto","or","other","others","otherwise","ought","our","ours","ourselves","out","outside","over","overall","own","p","particular",
                                "particularly","per","perhaps","placed","please","plus","possible","presumably","probably","provides","q","que","quite","qv","r",
                                "rather","rd","re","really","reasonably","regarding","regardless","regards","relatively","respectively","right","'s","said","same",
                                "saw","say","saying","says","second","secondly","see","seeing","seem","seemed","seeming","seems","seen","self","selves","sensible",
                                "sent","serious","seriously","seven","several","shall","she","should","shouldn't","since","six","so","some","somebody","somehow",
                                "someone","something","sometime","sometimes","somewhat","somewhere","soon","sorry","specified","specify","specifying","still","sub",
                                "such","sup","sure","t","t's","take","taken","tell","tends","th","than","thank","thanks","thanx","that","that's","thats","the","their",
                                "theirs","them","themselves","then","thence","there","there's","thereafter","thereby","therefore","therein","theres","thereupon","these",
                                "they","they'd","they'll","they're","they've","think","third","this","thorough","thoroughly","those","though","three","through",
                                "throughout","thru","thus","to","together","too","took","toward","towards","tried","tries","truly","try","trying","twice","two","u",
                                "un","under","unfortunately","unless","unlikely","until","unto","up","upon","us","use","used","useful","uses","using","usually","uucp",
                                "v","value","various","very","via","viz","vs","w","want","wants","was","wasn't","way","we","we'd","we'll","we're","we've","welcome",
                                "well","went","were","weren't","what","what's","whatever","when","whence","whenever","where","where's","whereafter","whereas","whereby",
                                "wherein","whereupon","wherever","whether","which","while","whither","who","who's","whoever","whole","whom","whose","why","will",
                                "willing","wish","with","within","without","won't","wonder","would","wouldn't","x","y","yes","yet","you","you'd","you'll","you're",
                                "you've","your","yours","yourself","yourselves","z","zero","html","ol"]

word_punct = set(stopwords.words('english')).union(punctuation).union(extended)

def preprocess(x):
    
    input = re.sub('[!@#$%^&*()\n_:><?\-.{}|+-,;""``~`—]|[0-9]|/|=|\[\]|\[\[\]\]',' ',x)
    input = re.sub('[“’\']','',input)  
    tmp = " "
    
    for i in word_tokenize(input):
        
        if i not in word_punct and len(i) > 3:
            tmp += i.lower() + " ";
    
    return tmp    
    

  


In [80]:
train_df['Script'] = train_df['Script'].map(preprocess)

In [81]:
train_df['Script'][0]

' crouch tiger hidden dragon wang ling jame schamu tsai jung base wang yuan compound secur porter load wagon convoy work lake lone horseman enter villag recogn worker master angl thirti power handsom background aunt sight drop parcel excitedli build yuan hallway aunt hurriedli hall aunt lien room beauti woman earli finish pack convoy wrap small item linen wrapper aunt burst aunt yuan practic hall aunt fine plea compo aunt usher carri larg object wrap silk smile long glanc room busi good fine awkward monk zheng wudan mountain practic deep medit mountain peac envi work busi rest left train earli wudan fighter train dure medit train place deep silenc surround light time space disappear place master told enlighten didnt feel bliss enlighten surround endless sorrow couldnt bear broke medit couldnt pull back someth leav prepar convoy deliveri peke perhap deliv unwrap object ancient astonishingli beauti sword green destini sword give greatest protector dont understand part pure blood wash eas

In [3]:
train_data = pk.load(open('processed_train_data','rb'))
test_data = pk.load(open('processed_test_data','rb'))

In [45]:
train_data = train_data.append(train_data[train_data['Labels'] == 18])
train_data.reset_index(drop=True, inplace=True)

### Let's Use LabelEncoder from scikit-learn to convert Genre labels to integers, 0, 1 2

In [46]:
from sklearn import preprocessing
lbl_enc = preprocessing.LabelEncoder()
index = lbl_enc.fit(train_data.Labels.values)

y = index.transform(train_data.Labels.values)


In [21]:
lbl_enc.classes_

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21], dtype=int64)

### Before going further it is important that we split the data into training and validation sets. We can do it using 
#### `train_test_split` from the `model_selection` module of scikit-learn.

In [42]:
train_test_split?

In [47]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train_data.processed_script.values, y, 
                                                    stratify = y,
                                                  test_size=0.30, shuffle=True)

In [51]:
print (xtrain.shape,ytrain.shape)
print (xvalid.shape)

(1385,) (1385,)
(594,)


# Building Basic Models

### Let's start building our very first model. 

### Our very first model is a simple TF-IDF (Term Frequency - Inverse Document Frequency) followed by a simple Logistic Regression.

In [52]:
# Always start with these features. They work (almost) everytime!
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv =  tfv.transform(xtrain) 
xvalid_tfv = tfv.transform(xvalid)

# count = CountVectorizer()


# count.fit(list(xtrain) + list(xvalid))

# xtrain_tfv =  count.transform(xtrain) 
# xvalid_tfv = count.transform(xvalid)


## TFIDF on test data

In [60]:
# x_test_tfv = tfv.transform(test_df['Script'].values)

In [53]:
## Fitting a simple Logistic Regression on TFIDF
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

import time
t1  = time.time()
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print(int(time.time() - t1))

203


In [63]:
# log_loss(predictions,yvalid)
# yvalid
# np.argmax(predictions,axis = 1)



log_loss(yvalid,clf.predict(xvalid_tfv))

ValueError: y_true and y_pred contain different number of classes 22, 2. Please provide the true labels explicitly through the labels argument. Classes found in y_true: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]

# Submission 

### Let's predict on the entire test data...

In [62]:
test_set_preds = pd.DataFrame(columns = train_df.Labels.unique().tolist())
test_set_preds.insert(0, 'File_Name', test_df.File_Name)

test_set_preds.head()

,File_Name,8,4,6,16,15,19,1,14,0,...,7,3,13,20,21,12,10,17,9,18
0,file_2300.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,file_809.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,file_1383.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,file_983.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,file_1713.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
test_set_preds.iloc[:,1:] = clf.predict_proba(x_test_tfv)

In [64]:
test_set_preds.shape

(849, 23)

In [65]:
test_set_preds.head()

,File_Name,8,4,6,16,15,19,1,14,0,...,7,3,13,20,21,12,10,17,9,18
0,file_2300.txt,0.0760497,0.048735,0.0152656,0.00442009,0.129339,0.0658912,0.200171,0.0157842,0.037981,...,0.00562854,0.010627,0.0405721,0.0698848,0.0468568,0.00441921,0.00439378,0.145499,0.0118471,0.00745845
1,file_809.txt,0.077653,0.0498873,0.0151105,0.00439055,0.143574,0.0687031,0.198798,0.0159996,0.0380866,...,0.00563151,0.0106764,0.0360964,0.106372,0.0459286,0.00438903,0.0043811,0.100615,0.0116716,0.00735454
2,file_1383.txt,0.0995415,0.10546,0.0158443,0.00459791,0.0959776,0.0588158,0.150394,0.0165801,0.0401404,...,0.00591484,0.0110333,0.0368607,0.0604629,0.095704,0.0046141,0.00460436,0.113418,0.0126151,0.00772805
3,file_983.txt,0.0886203,0.0543635,0.0152502,0.00454126,0.110677,0.0688275,0.178803,0.0164672,0.0390469,...,0.0058223,0.010992,0.0397705,0.0671665,0.050232,0.00455641,0.00456029,0.161282,0.0124211,0.00766134
4,file_1713.txt,0.125381,0.0804016,0.0162396,0.00458081,0.094498,0.059957,0.180637,0.0169377,0.041906,...,0.00583631,0.0111182,0.0497058,0.0589819,0.0541982,0.00460702,0.00459358,0.108324,0.012532,0.00777771


In [40]:
test_set_preds.head()

,File_Name,8,4,6,16,15,19,1,14,0,...,7,3,13,20,21,12,10,17,9,18
0,file_2300.txt,0.0756598,0.0486747,0.0151535,0.00442618,0.130199,0.0664058,0.193041,0.0157898,0.0375822,...,0.00564685,0.0106911,0.0391055,0.0710136,0.0476938,0.00442177,0.0044041,0.152639,0.0119087,0.00746575
1,file_809.txt,0.0783404,0.0500035,0.0150973,0.0044115,0.141571,0.0681948,0.198263,0.0162005,0.0380977,...,0.00565909,0.0107217,0.0364271,0.106151,0.0461993,0.00440567,0.00439849,0.10171,0.011686,0.0073923
2,file_1383.txt,0.103474,0.105138,0.0153343,0.00455124,0.0933858,0.0581633,0.163275,0.0164247,0.0382681,...,0.00583523,0.010894,0.0380384,0.058074,0.0936586,0.00455652,0.00455486,0.106823,0.012459,0.00766793
3,file_983.txt,0.0872246,0.0540455,0.0153065,0.00452159,0.109457,0.066747,0.181617,0.0162596,0.0385136,...,0.00579135,0.0109039,0.0392527,0.0668156,0.0495792,0.00453129,0.00452398,0.166751,0.0123774,0.00761597
4,file_1713.txt,0.124353,0.0806432,0.0160779,0.00457694,0.0943967,0.0589671,0.187166,0.0169017,0.0407518,...,0.00583984,0.0111348,0.0515907,0.0590796,0.0534829,0.00460397,0.00459076,0.105039,0.0124778,0.00775868


In [44]:
#Reorder the columns to match the Sample_submission_file
test_set_preds = test_set_preds[['File_Name',0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]]

In [52]:
test_set_preds.shape

(849, 23)

In [49]:
#Write your submissions to an excel file
test_set_preds.to_excel('test_set_preds_v1.xlsx', index=False)

#### Follow the sample submission format for your submission file

#### Please verify the following before submitting your solution to avoid an invalid sibmission.

1. The format of the file is excel(.xlsx)

2. The file doesn’t contain additional styling elements such as bold headings or table borders

3. The length of the submission exactly matches with that of the sample submission and test set

4. The file name doesnot have any spaces or special characters

5. All the columns are present

In [70]:
pd.read_excel('Movie_Scripts_Sample_Submission.xlsx')

,File_Name,0,1,2,3,4,5,6,7,8,...,12,13,14,15,16,17,18,19,20,21
0,file_2300.txt,0.254897,0.070870,0.034026,0.001088,0.076833,0.055085,0.108631,0.014625,0.040442,...,0.002126,0.007801,0.031488,0.047958,0.081665,0.001089,0.000549,0.103292,0.009705,0.004852
1,file_809.txt,0.072761,0.042652,0.011684,0.001046,0.128757,0.067255,0.234022,0.012940,0.034502,...,0.002092,0.007767,0.088627,0.070536,0.042840,0.001048,0.000524,0.114109,0.008936,0.004475
2,file_1383.txt,0.176885,0.055458,0.011611,0.000980,0.082053,0.046127,0.297048,0.011906,0.036915,...,0.001932,0.007315,0.030388,0.060440,0.041038,0.000975,0.000483,0.079213,0.009655,0.004332
3,file_983.txt,0.077208,0.040396,0.010318,0.000940,0.110868,0.069286,0.193717,0.011270,0.031018,...,0.001883,0.006810,0.031463,0.055402,0.040821,0.000947,0.000475,0.260130,0.008493,0.004113
4,file_1713.txt,0.108292,0.044440,0.010759,0.000963,0.081152,0.073162,0.174912,0.011675,0.032233,...,0.001917,0.007004,0.032439,0.050234,0.046141,0.000974,0.000484,0.264620,0.009009,0.004406
5,file_629.txt,0.074194,0.103399,0.012331,0.001050,0.138855,0.064591,0.223220,0.013726,0.035549,...,0.002067,0.007823,0.033660,0.069032,0.042212,0.001046,0.000523,0.106433,0.008985,0.004599
6,file_1213.txt,0.077656,0.040499,0.010044,0.000957,0.079574,0.060107,0.202646,0.011185,0.030428,...,0.001916,0.006881,0.035360,0.063224,0.043614,0.000978,0.000474,0.272704,0.008705,0.004181
7,file_2311.txt,0.060349,0.036126,0.010418,0.000927,0.190222,0.057193,0.294957,0.011681,0.030043,...,0.001810,0.006820,0.029462,0.074952,0.036173,0.000925,0.000459,0.099893,0.007935,0.003896
8,file_1004.txt,0.162405,0.063235,0.011864,0.001056,0.076578,0.066075,0.191650,0.013228,0.035822,...,0.002080,0.007673,0.032764,0.054917,0.072364,0.001057,0.000531,0.138742,0.010504,0.004873
9,file_1382.txt,0.177775,0.141331,0.014203,0.001109,0.082644,0.057721,0.110069,0.015034,0.091969,...,0.002196,0.008330,0.036050,0.053880,0.051163,0.001096,0.000557,0.091407,0.009518,0.004948
